#  HW1 NLP Anna Aksenova

В этом ноутбуке мы будем проводить анализ тональности на текстах из отзывов на IMDB о Гарри Поттере.

In [155]:
import re
from pprint import pprint
import time
import requests
from tqdm.auto import tqdm

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

Берем ссылку на IMDB

In [2]:
link = 'https://www.imdb.com/title/tt0241527/reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0 '

In [3]:
result = requests.get(link)
html = result.text

In [158]:
# эта функция открывает IMDB и кликает на кнопку, открывающую продолжение комментариев
def get_comments(url_add, maxclicks):
    
    progress_bar = tqdm(total = maxclicks)

    source_code = requests.get(url_add)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')


    driver = webdriver.Chrome('/Users/annaaksenova/Desktop/chromedriver')
    wait = WebDriverWait(driver, 100)

    driver.get(url_add)
    
    clicks = 0
    while True:
        clicks += 1
        if clicks <= maxclicks:
            more_button = wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "ipl-load-more__button"))).click()


        else:
            break
        progress_bar.update(1)
        
    time.sleep(25)
    source_code = driver.page_source

    plain_text = source_code
    soup = BeautifulSoup(plain_text, 'html.parser')

    database = []
    pageno = 1
    rating_list = []
    review_list = []

    for r in soup.find_all('div', {'class': 'lister-item-content'}):

        if "ipl-ratings-bar" in str(r):

            for rating in r.find_all('span', {'class': 'rating-other-user-rating'}):
                rating = str(rating.text).strip()
                if rating == '':
                    rating = 'NaN'
                rating_list.append(rating)

        for review in r.find_all('div', {'class': 'text'}):
            review = str(review.text)
            if review == '':
                review = 'NaN'
                review_list.append(review)
            else:
                review_list.append(review)
        progress_bar.close()
    return rating_list, review_list

In [159]:
rating_list, review_list = get_comments(link, 30)

Выберем комментарии, в которых проставлены оценки

In [81]:
com_rate = [(int(com[0].split('/')[0]), com[1]) for com in zip(rating_list, review_list) if not com[0].isalpha()]

Будем считать все оценки от 8 и выше положительными

In [82]:
positive = [com for com in com_rate if com[0]>=8]

Все оценки от 3 и ниже будем считать отрицательными

In [85]:
negative = [com for com in com_rate if com[0]<=3]

Посмотрим, сколько у нас вообще отзывов каждого типа

In [86]:
len(positive)

416

In [87]:
len(negative)

55

Сбалансируем выборку

In [88]:
positive_val = [(1, i) for i in positive[len(negative) - 10:len(negative)]]
negative_val = [(0, i) for i in negative[len(negative) - 10:]]

## Обработка текста

In [90]:
from collections import Counter 

import nltk
from nltk.stem import WordNetLemmatizer

In [91]:
lemmatizer = WordNetLemmatizer()

In [144]:
# эта функция собирает частотный словарь в 200 слов по дефолту
def collect_freqlist(reviews, max_len=200):
    
    freqlist = Counter()
    for text in reviews:
        for word in nltk.word_tokenize(text[1].lower()):
            if word.isalpha():
                word = lemmatizer.lemmatize(word)
                freqlist[word] += 1
    return dict(freqlist.most_common(max_len))

In [145]:
positive_list = collect_freqlist(positive[:len(negative) - 10])
negative_list = collect_freqlist(negative[:len(negative) - 10])

Создадим множества слов из "положительного" и "отрицательного" словарей и пересечем их

In [147]:
positive_list = set(positive_list.keys())
negative_list = set(negative_list.keys())
common = positive_list.intersection(negative_list)

In [148]:
positive_list.symmetric_difference_update(common)
negative_list.symmetric_difference_update(common)

## Определение типа отзыва по словам

In [150]:
def rate_detect(text):
    
    words = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(text[1][1].lower())]
    words = set(words)
    pos = len(words.intersection(positive_list))
    neg = len(words.intersection(negative_list))
    if pos > neg:
        check = 1
    else:
        check = 0
    return check

In [151]:
answers = []
for text in positive_val + negative_val:
    answers.append(rate_detect(text))

In [152]:
from sklearn.metrics import accuracy_score

In [153]:
print("Доля верных ответов:", accuracy_score(answers, [1 if i < 10 else 0 for i in range(20)]))

Доля верных ответов: 0.6


## Идеи улучшения

1. НОРМАЛЬНЫЙ лемматизатор, а не nltk, который распознает форму глагола прошедшего времени как существительное

2. Увеличить объем корпуса, слишком мало примеров негативных отзывов

3. Использовать другие признаки: смайлики, длину отзыва, упоминание книги (книголюбы склонны более негативно оценивать экранизацию)